In [ ]:
# import
import os
import sys
import pathlib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import tensorflow as tf
import keras
from keras import models
from keras import layers

In [ ]:
CONFIGURATION = {
    "IMAGE_HEIGHT": 160,  # 224 -> 160으로 축소 (계산량 50% 감소)
    "IMAGE_WIDTH": 160,
    "BATCH_SIZE": 64,  # 32 -> 64로 증가 (GPU 메모리 활용)
    "EPOCHS": 20,  # 30 -> 20으로 감소 (EarlyStopping으로 조기 종료)
    "LEARNING_RATE": 0.001,
    "NUM_CLASSES": 31,
    "DROPOUT_RATE": 0.3,  # 0.4 -> 0.3으로 감소
    "PATIENCE": 3,  # 5 -> 3으로 단축
    "VALIDATION_SPLIT": 0.2,
    "TRAIN_CSV": "dataset/train.csv",
    "TRAIN_IMAGE_DIR": "dataset/train/train",
    "TEST_CSV": "dataset/test.csv",
    "TEST_IMAGE_DIR": "dataset/test/test",
}

### PREPROCESSING
kaggle에서 다운로드한 데이터는 train.csv 와 train/train/\[images\].png 형태로 구분됨.
따라서, dataset으로 구성하기 위해서, paths와 labels 파싱하여
tf.data.Dataset.from_tensor_slices((paths, labels))를 구성한다


In [ ]:
# train csv 에서 ground_truth 불러오기
train_csv_path = pathlib.Path(CONFIGURATION["TRAIN_CSV"])
train_df = pd.read_csv(train_csv_path)
test_csv_path = pathlib.Path(CONFIGURATION["TEST_CSV"])
test_df = pd.read_csv(test_csv_path)
jaguar_names = train_df["ground_truth"].unique()
print(f"Shape of training data: {train_df.shape}")
print(f"Shape of testing data: {test_df.shape}")
print(f"Number of unique jaguar identities: {len(jaguar_names)}")

In [ ]:
train_image_set_dir = pathlib.Path(CONFIGURATION["TRAIN_IMAGE_DIR"])
test_image_set_dir = pathlib.Path(CONFIGURATION["TEST_IMAGE_DIR"])
existing_train = set(os.listdir(train_image_set_dir))
existing_test = set(os.listdir(test_image_set_dir))
print(f"Number of training images: {len(existing_train)}")
print(f"Number of testing images: {len(existing_test)}")

In [ ]:
### 클래스별 샘플 수 분석
class_distribution = train_df["ground_truth"].value_counts()
print("\n클래스별 샘플 분포:")
print(class_distribution)
print(f"\n평균 샘플 수: {class_distribution.mean():.2f}")
print(f"최대: {class_distribution.max()}, 최소: {class_distribution.min()}")

# 시각화
plt.figure(figsize=(12, 6))
class_distribution.plot(kind="bar")
plt.title("Class Distribution in Training Dataset")
plt.xlabel("Jaguar Identity")
plt.ylabel("Number of Samples")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
df = train_df[train_df["filename"].isin(existing_train)].copy()
df["path"] = df["filename"].apply(lambda x: os.path.join(train_image_set_dir, x))
class_names = sorted(df["ground_truth"].unique().tolist())
lookup = keras.layers.StringLookup(vocabulary=class_names, num_oov_indices=0)
paths = df["path"].tolist()
labels = df["ground_truth"].astype(str).tolist()
ds = tf.data.Dataset.from_tensor_slices((paths, labels))

# 데이터 증강 파이프라인
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        layers.RandomContrast(0.1),
    ]
)


def process_path(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(
        img, [CONFIGURATION["IMAGE_HEIGHT"], CONFIGURATION["IMAGE_WIDTH"]]
    )
    img = img / 255.0
    label = lookup(label)
    return img, label


def augment_path(img, label):
    img = data_augmentation(img, training=True)
    return img, label


ds = ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
ds = ds.shuffle(buffer_size=len(df))

# 데이터 증강은 학습 데이터에만 적용
train_size = int((1 - CONFIGURATION["VALIDATION_SPLIT"]) * len(df))
train_ds = ds.take(train_size)
val_ds = ds.skip(train_size)

train_ds = train_ds.map(augment_path, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(CONFIGURATION["BATCH_SIZE"]).prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.batch(CONFIGURATION["BATCH_SIZE"]).prefetch(tf.data.AUTOTUNE)

# 개선된 모델 아키텍처 (Dropout + BatchNormalization)
model = models.Sequential(
    [
        layers.InputLayer(
            input_shape=(CONFIGURATION["IMAGE_HEIGHT"], CONFIGURATION["IMAGE_WIDTH"], 3)
        ),
        layers.Conv2D(32, (3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(CONFIGURATION["DROPOUT_RATE"]),
        layers.Conv2D(64, (3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(CONFIGURATION["DROPOUT_RATE"]),
        layers.Conv2D(128, (3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(CONFIGURATION["DROPOUT_RATE"]),
        layers.Conv2D(256, (3, 3), activation="relu"),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(CONFIGURATION["DROPOUT_RATE"]),
        layers.Flatten(),
        layers.Dense(512, activation="relu"),
        layers.BatchNormalization(),
        layers.Dropout(CONFIGURATION["DROPOUT_RATE"]),
        layers.Dense(CONFIGURATION["NUM_CLASSES"], activation="softmax"),
    ]
)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=CONFIGURATION["LEARNING_RATE"]),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary()

In [ ]:
### 콜백 설정
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=CONFIGURATION["PATIENCE"],
    restore_best_weights=True,
    verbose=1,
)

reduce_lr = keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1
)

model_checkpoint = keras.callbacks.ModelCheckpoint(
    "jaguar_reid_best_model.h5", monitor="val_accuracy", save_best_only=True, verbose=1
)

# 모델 학습
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=CONFIGURATION["EPOCHS"],
    callbacks=[early_stopping, reduce_lr, model_checkpoint],
    verbose=1,
)

model.save("jaguar_reid_model.h5")

In [ ]:
### history 그래프 그리기
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.title("Accuracy over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.title("Loss over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
### Evaluate on test set with detailed metrics
test_df_filtered = test_df[test_df["filename"].isin(existing_test)].copy()
test_df_filtered["path"] = test_df_filtered["filename"].apply(
    lambda x: os.path.join(test_image_set_dir, x)
)
test_paths = test_df_filtered["path"].tolist()
test_labels = test_df_filtered["ground_truth"].astype(str).tolist()
test_ds = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
test_ds = test_ds.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(CONFIGURATION["BATCH_SIZE"]).prefetch(tf.data.AUTOTUNE)

test_loss, test_accuracy = model.evaluate(test_ds, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# 상세 평가 지표 (Classification Report)
y_pred_list = []
y_true_list = []

for images, labels in test_ds:
    predictions = model.predict(images, verbose=0)
    y_pred_list.extend(np.argmax(predictions, axis=1))
    y_true_list.extend(labels.numpy())

y_pred_array = np.array(y_pred_list)
y_true_array = np.array(y_true_list)

# 클래스별 성능 지표
print("\n=== Classification Report ===")
print(
    classification_report(
        y_true_array, y_pred_array, target_names=class_names, digits=4
    )
)

# 혼동 행렬 시각화
cm = confusion_matrix(y_true_array, y_pred_array)
plt.figure(figsize=(14, 12))
sns.heatmap(
    cm,
    annot=False,
    fmt="d",
    cmap="Blues",
    xticklabels=class_names,
    yticklabels=class_names,
)
plt.title("Confusion Matrix")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.xticks(rotation=45, ha="right")
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# 클래스별 정확도
per_class_accuracy = cm.diagonal() / cm.sum(axis=1)
print("\n=== Per-Class Accuracy ===")
for class_name, acc in zip(class_names, per_class_accuracy):
    print(f"{class_name}: {acc:.4f}")

In [ ]:
### 테스트 이미지 예측 시각화
def visualize_predictions(model, test_ds, num_samples=12):
    """테스트 셋에서 예측 결과를 시각화합니다."""
    plt.figure(figsize=(15, 10))

    sample_count = 0
    for images, labels in test_ds:
        for i in range(len(images)):
            if sample_count >= num_samples:
                break

            img = images[i].numpy()
            true_label_idx = labels[i].numpy()
            true_label = class_names[true_label_idx]

            # 예측
            prediction = model.predict(np.expand_dims(img, axis=0), verbose=0)
            pred_label_idx = np.argmax(prediction[0])
            pred_label = class_names[pred_label_idx]
            confidence = np.max(prediction[0])

            # 시각화
            plt.subplot(3, 4, sample_count + 1)
            plt.imshow(img)
            color = "green" if true_label == pred_label else "red"
            plt.title(
                f"True: {true_label}\nPred: {pred_label}\nConf: {confidence:.2f}",
                color=color,
            )
            plt.axis("off")

            sample_count += 1

        if sample_count >= num_samples:
            break

    plt.tight_layout()
    plt.show()


visualize_predictions(model, test_ds, num_samples=12)